# 第4回講義 宿題

## 課題

今Lessonで学んだことを元に，MNISTのファッション版 (Fashion MNIST，クラス数10) を多層パーセプトロンによって分類してみましょう．

Fashion MNISTの詳細については以下のリンクを参考にしてください．

Fashion MNIST: https://github.com/zalandoresearch/fashion-mnist

### 目標値

Accuracy 85%

### ルール

- 訓練データはx_train，t_train，テストデータはx_testで与えられます．
- 予測ラベルはone_hot表現ではなく0~9のクラスラベルで表してください．
- **下のセルで指定されているx_train、t_train以外の学習データは使わないでください**．
- Pytorchを利用して構いません．
- ただし，**torch.nn.Conv2dのような高レベルのAPIは使用しないで下さい**．具体的には，nn.Parameter, nn.Module, nn.Sequential以外のnn系のAPIです．
- torchvision等で既に実装されているモデルも使用しないで下さい．

### 提出方法

- 2つのファイルを提出していただきます．
  - テストデータ (x_test) に対する予測ラベルをcsvファイル (ファイル名: submission_pred.csv) で提出してください．
  - それに対応するpythonのコードをsubmission_code.pyとして提出してください (%%writefileコマンドなどを利用してください)．

### 評価方法

- 予測ラベルのt_testに対する精度 (Accuracy) で評価します．
- 即時採点しLeader Boardを更新します．（採点スケジュールは別アナウンス）
- 締切後の点数を最終的な評価とします．

### ドライブのマウント

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### データの読み込み（この部分は修正しないでください）

In [3]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import inspect
 
nn_except = ["Module", "Parameter", "Sequential"]
for m in inspect.getmembers(nn):
    if not m[0] in nn_except and m[0][0:2] != "__":
        delattr(nn, m[0]) 

seed = 1234
torch.manual_seed(seed)
np.random.seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

"""
#学習データ
x_train = np.load('drive/MyDrive/Colab Notebooks/DLBasics2023_colab/Lecture04/data/x_train.npy')
t_train = np.load('drive/MyDrive/Colab Notebooks/DLBasics2023_colab/Lecture04/data/y_train.npy')
    
#テストデータ
x_test = np.load('drive/MyDrive/Colab Notebooks/DLBasics2023_colab/Lecture04/data/x_test.npy')
"""
#学習データ
x_train = np.load('drive/MyDrive/Colab Notebooks/DeepLearning/Lecture04/data/x_train.npy')
t_train = np.load('drive/MyDrive/Colab Notebooks/DeepLearning/Lecture04/data/y_train.npy')
    
#テストデータ
x_test = np.load('drive/MyDrive/Colab Notebooks/DeepLearning/Lecture04/data/x_test.npy')



class train_dataset(torch.utils.data.Dataset):
    def __init__(self, x_train, t_train):
        self.x_train = x_train.reshape(-1, 784).astype('float32') / 255
        self.t_train = t_train

    def __len__(self):
        return self.x_train.shape[0]

    def __getitem__(self, idx):
        return torch.tensor(self.x_train[idx], dtype=torch.float), torch.tensor(self.t_train[idx], dtype=torch.long)

class test_dataset(torch.utils.data.Dataset):
    def __init__(self, x_test):
        self.x_test = x_test.reshape(-1, 784).astype('float32') / 255

    def __len__(self):
        return self.x_test.shape[0]

    def __getitem__(self, idx):
        return torch.tensor(self.x_test[idx], dtype=torch.float)

trainval_data = train_dataset(x_train, t_train)
test_data = test_dataset(x_test)

### 多層パーセプトロンの実装

In [4]:
batch_size = 32

val_size = 10000
train_size = len(trainval_data) - val_size

train_data, val_data = torch.utils.data.random_split(trainval_data, [train_size, val_size])

dataloader_train = torch.utils.data.DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_valid = torch.utils.data.DataLoader(
    val_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_test = torch.utils.data.DataLoader(
    test_data,
    batch_size=batch_size,
    shuffle=False
)

In [5]:
def relu(x):
    x = torch.where(x>0, x, torch.zeros_like(x))
    return x


def softmax(x):
    x -= torch.cat([x.max(axis=1, keepdim=True).values] * x.size()[1], dim=1)
    x_exp = torch.exp(x)
    return  x_exp/torch.cat([x_exp.sum(dim=1, keepdim=True)] * x.size()[1], dim=1)


class Dense(nn.Module):  # nn.Moduleを継承する
    def __init__(self, in_dim, out_dim, function=lambda x: x):
        super().__init__()
        self.W = nn.Parameter(torch.tensor(np.random.uniform(
                        low=-np.sqrt(6/in_dim),
                        high=np.sqrt(6/in_dim),
                        size=(in_dim, out_dim)
                    ).astype('float32')))
        self.b = nn.Parameter(torch.tensor(np.zeros([out_dim]).astype('float32')))
        self.function = function

    def forward(self, x):
        return self.function(torch.matmul(x, self.W) + self.b)




class MLP(nn.Module):  # nn.Moduleを継承する
    def __init__(self, in_dim, hid_dim, out_dim):
        super(MLP, self).__init__()
        self.linear1 = Dense(in_dim, hid_dim, function=relu)
        self.linear2 = Dense(hid_dim, out_dim, function=softmax)
      
    def forward(self, x):
        x = self.linear1(x)
        x = self.linear2(x)
        return x





In [13]:
in_dim = 784
hid_dim = 200
out_dim = 10
lr = 0.00005
n_epochs = 100


#mlp = MLP(in_dim,hid_dim, out_dim).to(device)
mlp = nn.Sequential(
    Dense(784, 500, relu),
    Dense(500, 200, relu),
    Dense(200, 128, relu),
    Dense(128, 10, softmax)
).to(device)


optimizer = optim.Adam(mlp.parameters(), lr=lr)

for epoch in range(n_epochs):
    losses_train = []
    losses_valid = []
    train_num = 0
    train_true_num = 0
    valid_num = 0
    valid_true_num = 0

    mlp.train()
    for x, t in dataloader_train:
        true = t.tolist()

        t_hot = torch.eye(10)[t]
        x = x.to(device)
        t_hot = t_hot.to(device)
        y = mlp.forward(x)
        loss = -(t_hot*torch.log(y)).sum(axis=1).mean()

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

        pred = y.argmax(1)        

        losses_train.append(loss.tolist())

        acc = torch.where(t - pred.to("cpu") == 0, torch.ones_like(t), torch.zeros_like(t))
        train_num += acc.size()[0]
        train_true_num += acc.sum().item()

    mlp.eval()
    for x, t in dataloader_valid:
        true = t.tolist()

        t_hot = torch.eye(10)[t]

       
        x = x.to(device)
        t_hot = t_hot.to(device)

        
        y = mlp.forward(x)

        
        loss = -(t_hot*torch.log(y)).sum(axis=1).mean()

        
        pred = y.argmax(1)

        losses_valid.append(loss.tolist())

        acc = torch.where(t - pred.to("cpu") == 0, torch.ones_like(t), torch.zeros_like(t))
        valid_num += acc.size()[0]
        valid_true_num += acc.sum().item()

    if np.mean(losses_train) < 0.05:
      print("loss is too small")
      break

    print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}]'.format(
        epoch,
        np.mean(losses_train),
        train_true_num/train_num,
        np.mean(losses_valid),
        valid_true_num/valid_num
    ))

EPOCH: 0, Train [Loss: 0.640, Accuracy: 0.790], Valid [Loss: 0.461, Accuracy: 0.840]
EPOCH: 1, Train [Loss: 0.433, Accuracy: 0.850], Valid [Loss: 0.409, Accuracy: 0.853]
EPOCH: 2, Train [Loss: 0.391, Accuracy: 0.862], Valid [Loss: 0.382, Accuracy: 0.865]
EPOCH: 3, Train [Loss: 0.363, Accuracy: 0.872], Valid [Loss: 0.360, Accuracy: 0.869]
EPOCH: 4, Train [Loss: 0.342, Accuracy: 0.879], Valid [Loss: 0.344, Accuracy: 0.876]
EPOCH: 5, Train [Loss: 0.325, Accuracy: 0.885], Valid [Loss: 0.350, Accuracy: 0.872]
EPOCH: 6, Train [Loss: 0.310, Accuracy: 0.889], Valid [Loss: 0.332, Accuracy: 0.880]
EPOCH: 7, Train [Loss: 0.298, Accuracy: 0.893], Valid [Loss: 0.319, Accuracy: 0.884]
EPOCH: 8, Train [Loss: 0.287, Accuracy: 0.897], Valid [Loss: 0.333, Accuracy: 0.877]
EPOCH: 9, Train [Loss: 0.277, Accuracy: 0.901], Valid [Loss: 0.310, Accuracy: 0.888]
EPOCH: 10, Train [Loss: 0.267, Accuracy: 0.903], Valid [Loss: 0.322, Accuracy: 0.882]
EPOCH: 11, Train [Loss: 0.258, Accuracy: 0.907], Valid [Loss: 0.

In [14]:
mlp.eval()

t_pred = []
for x in dataloader_test:

    x = x.to(device)

    # 順伝播
    y = mlp.forward(x)

    # モデルの出力を予測値のスカラーに変換
    pred = y.argmax(1).tolist()

    t_pred.extend(pred)

submission = pd.Series(t_pred, name='label')
submission.to_csv('drive/MyDrive/Colab Notebooks/DeepLearning/Lecture04/submission_pred.csv', header=True, index_label='id')